In [11]:
import nltk
import pandas as pd
nltk.download('punkt',quiet=True)
nltk.download("stopwords",quiet=True)
import string 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pickle

class Query:
    input_li = []
    # op_li = []

    def tokenize_seq(self,new_s):
        new_s = new_s.lower()
        translate_table = dict((ord(char), " ") for char in string.punctuation)   
        new_s = new_s.translate(translate_table)
        li = word_tokenize(new_s)
        stop_words = set(stopwords.words("english"))
        filter_li = []
        for words in li:
            if(words not in stop_words):
                filter_li.append(words)
        return filter_li

    def __init__(self,input_seq):
        input_li = self.tokenize_seq(input_seq)
        # op_li = []
        # first_op_li = []
        # if(op_seq != ""):
        #     first_op_li = op_seq.split(",")
        #     for i in range(len(first_op_li)):
        #         temp_l = first_op_li[i].split()
        #         for j in temp_l:
        #             op_li.append(j)
        # for i in range(len(op_li)):
        #     op_li[i] = (op_li[i].strip()).lower()
        self.input_li = input_li
        # self.op_li = op_li
    
    def getQuery(self):
        return self.input_li

class Positional_Inverted_Index:
    inverted_ind = {} # {word:dict} #dict{id:list}
    document_count = 0
    name_arr = []
    word_freq = {}

    # def extract_text(self,s,find_tags = False, remove_separators = False):
    #     new_s = s[::]
    #     if(find_tags):
    #         start = [0,0] #Start is inclusive 
    #         end = [0,0] #End is exclusive
    #         for i in range(len(s)):
    #             if(i + 7 <= len(s) and s[i:i+7] == "<TITLE>"):
    #                 start[0] = i+7
    #             elif(i + 8 <= len(s) and s[i:i+8] == "</TITLE>"):
    #                 end[0] = i
    #             elif(i + 6 <= len(s) and s[i:i+6] == "<TEXT>"):
    #                 start[1] = i+6
    #             elif(i + 7 <= len(s) and s[i:i+7] == "</TEXT>"):
    #                 end[1] = i
    #         new_s = s[start[0]:end[0]] + " " + s[start[1]:end[1]]
    #     if(remove_separators):
    #         new_s = " ".join(new_s.split("\n"))
    #         new_s = " ".join(new_s.split("-"))
    #     return new_s
    
    def __init__(self):
        self.inverted_ind = {}
        self.document_count = 0
        self.name_arr = []
        self.word_freq = {}

    def new_Data(self,ind,text):
        # f = open(path,"r")
        # s = f.read()
        # f.close()
        # new_s = self.extract_text(s)
        new_s = text
        new_s = new_s.lower()
        
        translate_table = dict((ord(char), " ") for char in string.punctuation)   
        new_s = new_s.translate(translate_table)
        li = word_tokenize(new_s)
        stop_words = set(stopwords.words("english"))
        filter_li = []
        for words in li:
            if(words not in stop_words):
                filter_li.append(words)
        filter_li = list(set(filter_li))
        self.addDoc(filter_li,int(ind))
        # self.name_arr.append(path)
        # self.document_count+=1
    
    def addDoc(self,token_list,doc_id): #token list in order, id of parent document 
        for i in range(len(token_list)):
            key = token_list[i]
            if(key in self.inverted_ind.keys()):
                self.word_freq[key]+=1
                dict = self.inverted_ind[str(key)]
                if(int(doc_id) not in dict.keys()):
                    dict[int(doc_id)] = [i]
                else:
                    dict[int(doc_id)].append(i)
            else:
                self.word_freq[key] = 1
                dict = {}
                dict[int(doc_id)] = [i]
                self.inverted_ind[str(key)] = dict

    def getPositionalList(self,key):
        return self.inverted_ind[str(key)]
        
    def binary_search(self,arr, x):
        low = 0
        high = len(arr) - 1
        mid = 0
        while low <= high:
            mid = (high + low) // 2
            if arr[mid] < x:
                low = mid + 1
            elif arr[mid] > x:
                high = mid - 1
            else:
                return 1
        return 0
    
    def getFreq(self,key):
        return len(self.inverted_ind[key])
    
    def check_existence(self,word,document_id,position):
        if word in self.inverted_ind.keys():
            if document_id in self.inverted_ind[word].keys():
                return self.binary_search(self.inverted_ind[word][document_id],position)
            else:
                return -1
        return -2

    def processHelper(self,input_li):
        output = []
        override = 1
        if input_li[0] not in self.inverted_ind.keys():
            return output
        for cur_doc in self.inverted_ind[input_li[0]].keys():
            for cur_pos in self.inverted_ind[input_li[0]][cur_doc]:
                next_pos = cur_pos+1
                j = 1
                while(j< len(input_li)):
                    x = self.check_existence(input_li[j],cur_doc,next_pos)
                    j+=1
                    next_pos+=1
                    override = x
                    if(override <= 0):
                        break
                if(override <= -1):
                    break
                if(j == len(input_li)):
                    output.append(cur_doc)
            if(override <= -2):
                break
        return output
    
    def getOutput(self,input_seq,show_names = False):
        query = Query(input_seq)
        input_li= query.getQuery()
        if(len(input_li) > 0):
            input_li.append(input_li[-1])
        output = list(set(self.processHelper(input_li)))
        output.sort()
        # if(show_names):
        #     out_list = []
        #     for i in output:
        #         out_list.append(self.name_arr[i])
        #     return out_list
        return output

def getNum(i):
    return (4-len(str(i)))*"0" + str(i)


try:
    invertedIndex = pickle.load(open("baseline2_savefile.pickle", "rb"))
    print(invertedIndex.inverted_ind['jurassic'])
    print(invertedIndex.inverted_ind['world'])
except (OSError, IOError) as e:
    invertedIndex = Positional_Inverted_Index()
    df = pd.read_csv('merged_genre.csv')

    for ind in df.index:
        f = invertedIndex.new_Data(ind, str(df['title'][ind]) + " " + str(df['cast'][ind]) + " " + str(df['listed_in'][ind]) + " " + str(df['description'][ind]))
    # for j in range(1,1401): # 1,1401
    #     f = inverted_index.new_Data("Data/CSE508_Winter2023_Dataset/cranfield"+getNum(j))
    pickle.dump(invertedIndex, open("baseline2_savefile.pickle", "wb"))

print("\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
df = pd.read_csv('merged_genre.csv')
n = int(input())

for i in range(n):
    inp_seq = input()
    print("Search Phrase:",inp_seq)
    output = invertedIndex.getOutput(inp_seq,show_names = True)
    print("Number of Documents Retrieved:",len(output))
    
    if(len(output)>0):
        print("Movie recommendations:")
        for j in range(len(output)):
            print("\t",str(j+1)+".",df['title'][int(output[j])])
    else:
        print("No results found!")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

{464: [5], 7089: [5], 7990: [3], 7991: [5]}
{32: [23], 53: [9], 57: [16], 66: [11], 70: [17], 90: [28], 103: [29], 110: [28], 116: [7], 121: [23], 137: [35], 141: [23], 165: [15], 200: [7], 208: [30], 215: [17], 230: [25], 250: [9], 253: [26], 254: [21], 272: [23], 284: [35], 294: [22], 314: [20], 334: [21], 351: [10], 354: [19], 360: [19], 368: [48], 394: [20], 397: [18], 407: [25], 414: [31], 446: [16], 448: [10], 454: [11], 464: [24], 470: [14], 539: [30], 542: [9], 567: [29], 589: [14], 630: [32], 662: [35], 697: [21], 713: [21], 730: [14], 742: [36], 747: [7], 754: [23], 755: [13], 765: [8], 767: [18], 769: [27], 775: [16], 825: [30], 826: [25], 846: [21], 847: [16], 855: [16], 860: [16], 877: [9], 886: [15], 903: [26], 921: [26], 927: [23], 932: [16], 933: [16], 938: [13], 952: [14], 957: [27], 981: [24], 985: [10], 1010: [13], 1034: [19], 1056: [15], 1058: [30], 1061: [20], 1065: [27], 1070: [27], 1089: [23], 1096: [32], 1099: [34], 1102: [28], 1140: [14], 1145: [16], 1193: [26]

ValueError: invalid literal for int() with base 10: ''